In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth',500)
import numpy as np
import _pickle as pickle


In [2]:
with open("profiles_clustered.pkl",'rb') as fp:
    df = pickle.load(fp)

with open("profilesMapIndex.pkl",'rb') as fp:
    profileMap = pickle.load(fp)


In [4]:
df

AttributeError: 'DataFrame' object has no attribute '_data'

In [46]:
df = df.drop('Cluster #', axis=1)

In [47]:
new_profile = pd.DataFrame(columns=df.columns)

In [48]:
temp_list = ['0', '0', '0', '0', '1', '1', '9', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '9', '0', '1', '1', '0', '0', '0', '1', '9', '0', '0', '0', '1', '1', '0', '4']
len(temp_list)

33

In [49]:
data_to_append = {}
for i in range(len(new_profile.columns)):
    data_to_append[new_profile.columns[i]] = temp_list[i]
new_profile = new_profile.append(data_to_append, ignore_index = True)

In [50]:
s_scores

[0.5431847214645259]

In [51]:
new_profile

,biography,detective,fiction,mystery,non fiction,novel,Book,action,bollywood,comedy,...,pop,rock,Music,baseball,basketball,chess,cricket,football,hockey,Sport
0,0,0,0,0,1,1,9,0,0,1,...,0,1,9,0,0,0,1,1,0,4


In [52]:
new_profile.index = [df.index[-1] + 1]

In [53]:
df=df.append(new_profile)

In [54]:
new_profile_map = pd.DataFrame(columns=profileMap.columns)
new_profile_map['uid'] = [193]
new_profile_map.index=new_profile.index
profileMap = profileMap.append(new_profile_map)
#profileMap = profileMap.append([[new_profile.index,221]])

In [55]:
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df),columns=df.columns[0:],index=df.index)

cluster_cnt = [i for i in range(2, 3, 1)]


s_scores = []

db_scores = []


for i in cluster_cnt:
    
    
    hac = AgglomerativeClustering(n_clusters=i)
    
    hac.fit(df)
    
    cluster_assignments = hac.labels_
    
    s_scores.append(silhouette_score(df, cluster_assignments))
    
    db_scores.append(davies_bouldin_score(df, cluster_assignments))



In [56]:
def cluster_eval(y, x, z):
    """
    Prints the scores of a set evaluation metric. Prints out the max and min values of the evaluation scores.
    """
    
    # Creating a DataFrame for returning the max and min scores for each cluster
    df = pd.DataFrame(columns=['Cluster Score'], index=[i for i in range(2, len(y)+2)])
    df['Cluster Score'] = y
    
    print('Max Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].max()])
    print('\nMin Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].min()])
    
    if z==1:
        return df[df['Cluster Score']==df['Cluster Score'].max()]
    else:
        return df[df['Cluster Score']==df['Cluster Score'].min()]
    
    
print("\nThe Silhouette Coefficient Score (find max score):")
effi_cluster = (int)((cluster_eval(s_scores, cluster_cnt,1).index[0] + cluster_eval(db_scores, cluster_cnt,2).index[0])/2)

print("\nThe Davies-Bouldin Score (find minimum score):")



The Silhouette Coefficient Score (find max score):
Max Value:
Cluster #    Cluster Score
2       0.529108

Min Value:
Cluster #    Cluster Score
2       0.529108
Max Value:
Cluster #    Cluster Score
2       0.289808

Min Value:
Cluster #    Cluster Score
2       0.289808

The Davies-Bouldin Score (find minimum score):


In [57]:
effi_cluster

2

In [58]:
hac = AgglomerativeClustering(n_clusters=effi_cluster)


hac.fit(df)


cluster_assignments = hac.labels_


df = (pd.DataFrame(scaler.inverse_transform(df), columns=df.columns[0:], index=df.index))


df['Cluster #'] = cluster_assignments


df

,biography,detective,fiction,mystery,non fiction,novel,Book,action,bollywood,comedy,...,rock,Music,baseball,basketball,chess,cricket,football,hockey,Sport,Cluster #
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,1
1,0.0,0.0,0.0,0.0,1.0,1.0,0.012346,0.0,0.0,1.0,...,1.0,0.012346,0.0,0.0,0.0,1.0,1.0,0.0,0.0625,0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.111111,0.0,0.0,1.0,...,1.0,0.111111,0.0,0.0,0.0,1.0,1.0,0.0,0.2500,0
3,0.0,0.0,0.0,0.0,1.0,1.0,1.000000,0.0,0.0,1.0,...,1.0,1.000000,0.0,0.0,0.0,1.0,1.0,0.0,1.0000,0
4,0.0,0.0,0.0,0.0,1.0,1.0,9.000000,0.0,0.0,1.0,...,1.0,9.000000,0.0,0.0,0.0,1.0,1.0,0.0,4.0000,0
5,0.0,0.0,0.0,0.0,1.0,1.0,9.000000,0.0,0.0,1.0,...,1.0,9.000000,0.0,0.0,0.0,1.0,1.0,0.0,4.0000,0


In [60]:
with open("profiles_clustered.pkl","wb") as fp:
    pickle.dump(df,fp)
    
with open("profilesMapIndex.pkl","wb") as fp:
    pickle.dump(profileMap,fp)